In [ ]:
!pip install selenium
!pip install python-dotenv
!pip install pytest-playwright
!playwright install
!pip install nest_asyncio

In [ ]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright

nest_asyncio.apply()

async def scrape_website(url, next_button_class, element_to_check):
    count = 0
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url)

        while True:
            await page.wait_for_selector('body')
            page_source = await page.content()

            count += 1
            print(count)
            # print(page_source)

            try:
                next_button = page.locator(f'.{next_button_class}')
                if await next_button.count() > 0:
                    await next_button.click()
                    await page.wait_for_load_state('networkidle')  # Wait until the next page is loaded
                    await page.wait_for_selector(element_to_check)  # Wait for specific element to be visible
                    await page.mouse.click(x=0, y=page.viewport_size['height'] // 2)  # Perform the additional action
                else:
                    print("No more pages.")
                    break
            except Exception as e:
                print(f"An error occurred: {e}")
                break

        await browser.close()

async def main():
    await scrape_website('https://2717recovery.com/products/recovery-cream', 'jdgm-paginate__next-page', 'body')
    # await scrape_website('https://bhumi.com.au/products/organic-cotton-blissful-ritual-twill-bath-sheet-bundle', 'yotpo-reviews-pagination-item', 'body')

asyncio.run(main())
